In [1]:
# Import packages
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
import matplotlib.pyplot as plt
import scipy.stats
import phi_divergence as phi
import time

In [2]:
# Matplotlib settings:
plt.rcParams['figure.figsize'] = [9, 7]
plt.rcParams['figure.dpi'] = 100 # can be increased for better quality

The problem we examine is as follows:

\begin{align}
\label{math_form:examples:pm2}
    \max_{\mathbf{x}}&~\theta \\
    \text{s.t.}&~\mathbf{r}^T \mathbf{x} \geq \theta \\
    &~\mathbf{e}^T \mathbf{x} = 1, \\
    &~\mathbf{x} \geq 0,
\end{align}

where $\mathbf{x}, \mathbf{r} \in \mathbb{R}^{k}$

We randomly sample $N$ returns for k assets, which is done in the following way:

\begin{equation}
\tilde{r}_{i}=\left\{\begin{array}{ll}
\frac{\sqrt{\left(1-\gamma_{i}\right) \gamma_{i}}}{\gamma_{i}} & \text { with probability } \gamma_{i} \\[2mm]
-\frac{\sqrt{\left(1-\gamma_{i}\right) \gamma_{i}}}{1-\gamma_{i}} & \text { with probability } 1-\gamma_{i}
\end{array}, \quad \gamma_{i}=\frac{1}{2}\left(1+\frac{i}{k + 1}\right), \quad i=1, \ldots, k. \right.
\end{equation}

In [3]:
# Problem specific functions:
def generate_data(k, N, N_test):
    data_nominal = np.array([[0] * k]) # Always assigned to train data
    np.random.seed(1)
    data = np.random.uniform(-1,1,size = (N-1,k)) # generate N-1 scenarios

    # Randomly assign N_test from N-1 to test data
    np.random.shuffle(data) # don't think this is really necessary, but just in case...
    data_test, data_train = data[:N_test,:], data[N_test:,:]

    # add nominal case to training data
    data = np.concatenate((data,data_nominal)) 
    data_train = np.concatenate((data_train,data_nominal))
    
    return data, data_nominal, data_test, data_train 

def toymodel(Z_arr, k, time_limit):
    x = cp.Variable(k, nonneg = True)
    constraints = [Z_arr @ x <= 1, cp.sum(x[0:(k-1)]) <= x[k-1]-1, x<=10]
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    return(x.value, prob.value)

def get_true_prob(x, k):
    return(1/2+1/(2*x[k-1]))
    
def toymodel_true_prob(beta, k):
    x = cp.Variable(k, nonneg = True)
    constraints = [(1-2*beta)*x[k-1] + 1 >= 0, cp.sum(x[0:(k-1)]) <= x[k-1]-1, x<=10]
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(x.value, prob.value)

In [ ]:
# Auxillary functions:
def compute_opt_given_data(alpha, beta, par, phi_div, data, time_limit_mosek):
    N = data.shape[0]
    M = 1000 # set M large enough such that constraint is made redundant
    p_min, lb = determine_min_p(alpha, beta, par, phi_div, N)
    
    if p_min > 1:
        return None, None, None, None, p_min
    
    F = np.ceil(p_min * N)
    start_time = time.time()
    x, y, obj = opt_set(data, F, M, time_limit_mosek)
    runtime = time.time() - start_time
    sum_y = np.sum(y)
    return runtime, x, sum_y, obj, p_min

def opt_set(data, F, M, time_limit):
    N = data.shape[0]
    k = data.shape[1]
    x = cp.Variable(k, nonneg = True)
    y = cp.Variable(N, boolean = True)
    constraints = [cp.sum(x[0:(k-1)]) <= x[k-1]-1, x <= 10, data @ x <= 1 + (1-y)*M, cp.sum(y) >= F]
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    return(x.value, y.value, prob.value)

def determine_min_p(alpha, beta, par, phi_div, N):
    # "fixed" settings for this procedure
    delta = 0.1
    epsilon = 0.0001
    r = phi_dot/(2*N)*scipy.stats.chi2.ppf(1-alpha, 1)
    p = np.array([beta, 1-beta])
    lb = lowbound(p, r, par, phi_div)
    p_prev = p
    while True:
        if p[0] + delta > 1 - epsilon:
            delta = delta/10
        p = p + np.array([delta, -delta])
        lb = lowbound(p, r, par, phi_div)
        if lb < beta:
            continue
        else:
            delta = delta / 10
            if delta < epsilon:
                break
            else:
                p = p_prev 
    return p[0], lb

def check_conv_comb(Z_arr):
    conv_comb_points = []
    if len(Z_arr) >= 3:
        for i in range(len(Z_arr)):
            z_i = Z_arr[i]
            Z_rest = np.append(Z_arr[:i], Z_arr[(i+1):], axis = 0)
            # solve optimization problem, if feasible, z_i is convex combination of points in Z_rest 
            # (https://en.wikipedia.org/wiki/Convex_combination)
            alpha = cp.Variable(len(Z_rest), nonneg = True)
            constraints = [alpha @ Z_rest == z_i, cp.sum(alpha) == 1]
            obj = cp.Maximize(alpha[0]) # no true objective function, only interested whether feasible solution exists
            prob = cp.Problem(obj,constraints)
            prob.solve(solver=cp.MOSEK)
            if prob.status != 'infeasible':
                conv_comb_points.append(i) 
    return conv_comb_points

def compute_calafiore_N_min(dim_x, beta, alpha):
    return np.ceil(dim_x /((1-beta)*alpha)).astype(int)

In [4]:
# Methodology-related functions:
def lowbound(p, r, par, phi_div):
    q = cp.Variable(2, nonneg = True)
    constraints = [cp.sum(q) == 1]
    constraints = phi_div(p,q,r,par,constraints)
    obj = cp.Minimize(q[0])
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(prob.value)

def solve_with_alg(add_strategy, remove_cc, remove_nonbinding, 
                   alpha, beta, par, phi_div, numeric_precision,
                   data_train, data_test, data_nominal, 
                   time_limit_mosek, time_limit_alg):
    # Get extra info
    N_train = len(data_train)
    N_test = len(data_test)
    k = data_train.shape[1]
    r = phi_dot/(2*N_test)*scipy.stats.chi2.ppf(1-alpha, 1)
    
    # initialize values
    Z_arr = data_nominal
    lb = -np.inf
    num_iter = 0
    
    start_time = time.time()
    while True:
        [x, obj] = toymodel(Z_arr, k, time_limit_mosek)
        
        constr_test = data_test.dot(x)
        vio_test = constr_test[constr_test>(1+numeric_precision)]
        constr_train = data_train.dot(x)
        vio_train = constr_train[constr_train>(1+numeric_precision)]
    
        if (len(vio_train) == 0 or len(vio_test) == 0):
            break        
        
        p = np.array([(N_test-len(vio_test))/N_test, len(vio_test)/N_test])
        lb = lowbound(p, r, par, phi_div)
        
        if (lb > beta or (time.time() - start_time) > time_limit_alg):
            break
        
        # Addition strategies
        Z_to_add = None
        if add_strategy == 'smallest_vio':   # the least violated scenario is added
            vio_min = np.min(vio_train)        
            ind = np.where(constr_train == vio_min)[0][0]
            Z_to_add = np.array([data_train[ind]])
        elif add_strategy == 'N*(beta-lb)_smallest':   # the N*(beta-lb)-th scenario is added
            rank = np.ceil(N_train*(beta-lb)).astype(int)
            vio_sort = np.sort(vio_train) 
            vio_value = vio_sort[rank-1]     # -1 to correct for python indexing
            ind = np.where(constr_train == vio_value)[0][0]  
            Z_to_add = np.array([data_train[ind]])
        elif add_strategy == 'random':
            vio_rand = np.random.choice(vio_train)
            ind = np.where(constr_train == vio_rand)[0][0]   
            Z_to_add = np.array([data_train[ind]])
        else:
            print("Error: did not provide valid addition strategy")
            break
        
        Z_arr = np.append(Z_arr, Z_to_add, axis = 0)
        
        # Removal strategies
        if remove_cc:
            conv_combs = check_conv_comb(Z_arr)
            Z_arr = np.delete(Z_arr, conv_combs, axis=0)
            count_removed += len(conv_combs)
        
        #if remove_nonbinding:
            #TODO:....
        
        
        num_iter += 1        
            
    runtime = time.time() - start_time
    true_prob = get_true_prob(x, k)
    return runtime, x, obj, lb, true_prob, num_iter, Z_arr  

In [5]:
# Display output functions:
def plot_iter(name, num_iter, data, Z_arr, x, obj, lb, save_plot, plot_type, show_legend):
    plt.plot(data[:,0],data[:,1],'ok',markersize=1, label = 'All scenarios')
    
    if Z_arr is not None:
        plt.plot(Z_arr[:,0],Z_arr[:,1], color='blue', marker='+', linestyle='',
                 markersize=10, label = 'Chosen scenarios')

    # Add constraint to plot, given solution x
    constraint_x = np.linspace(-1, 1, 1000)
    constraint_y = (1 - x[0]*constraint_x) / x[1]
    plt.plot(constraint_x, constraint_y, '--r', label = r'$\xi_{1}x_{1}^{*}+\xi_{2}x_{2}^{*}\leq 1$' ,alpha=1)

    plt.title('Iteration '+str(num_iter)+': Solution = (' + str(round(x[0],3)) + ', ' 
              + str(round(x[1],3)) + '), Objective value = ' + str(round(obj,3)) 
              + ', Lower bound = '+ str(round(lb,3)))
    plt.xlabel(r'$\xi_1$')
    plt.ylabel(r'$\xi_2$')
    
    if show_legend:
        plt.legend(bbox_to_anchor=(1.01, 0.6), loc='upper left')
    
    plt.tight_layout()
    
    if save_plot:
        plot_name = 'Figures/ToyModel/Scenarios_wConstraint_iter='+str(num_iter)+'_N=' + str(N) + '_alpha=' + str(alpha) + "_beta="+ str(beta)
        plt.savefig(plot_name + '.' + plot_type)
    
    plt.show()
    
def plot_solution(name, data, Z_arr, x, obj, lb, save_plot, plot_type, show_legend):
    if data.shape[1] > 2:
        print("ERROR: Cannot print larger than 2 dim")
        return
    
    plt.plot(data[:,0],data[:,1],'ok',markersize=1, label = 'All scenarios')
    
    if Z_arr is not None:
        plt.plot(Z_arr[:,0],Z_arr[:,1], color='blue', marker='+', linestyle='',
                 markersize=10, label = 'Chosen scenarios')

    # Add constraint to plot, given solution x
    constraint_x = np.linspace(-1, 1, 1000)
    constraint_y = (1 - x[0]*constraint_x) / x[1]
    plt.plot(constraint_x, constraint_y, '--r', label = r'$\xi_{1}x_{1}^{*}+\xi_{2}x_{2}^{*}\leq 1$' ,alpha=1)

    plt.title(name +': $\mathbf{x}^{*}$ = (' + f'{round(x[0],3):.3f}'
              + ', ' + f'{round(x[1],3):.3f}'
              + '), Obj = ' + f'{round(obj,3):.3f}'
              + ', LB = '+ f'{round(lb,3):.3f}')
    
    plt.xlabel(r'$\xi_1$')
    plt.ylabel(r'$\xi_2$')
    
    if show_legend:
        plt.legend(bbox_to_anchor=(1.01, 0.6), loc='upper left')
    
    plt.tight_layout()
    
    if save_plot:
        plot_name = 'Figures/ToyModel/Scenarios_wConstraint_'+name+'_N=' + str(N) + '_alpha=' + str(alpha) + "_beta="+ str(beta)
        plt.savefig(plot_name + '.' + plot_type)
    
    plt.show()
    
def write_output_to_latex(num_settings, headers, data):
    textabular = f"{'l'*num_settings}|{'r'*(len(headers)-num_settings)}"
    texheader = " & ".join(headers) + "\\\\"
    texdata = "\\hline\n"
    for label in data:
        if num_settings == 1:
            texdata += f"{label} & {' & '.join(map(str,data[label]))} \\\\\n"
        elif num_settings == 2:
            texdata += f"{label[0]} & {label[1]} & {' & '.join(map(str,data[label]))} \\\\\n"
        elif num_settings == 3:
            texdata += f"{label[0]} & {label[1]} & {label[2]} & {' & '.join(map(str,data[label]))} \\\\\n"
        else:
            print("ERROR: provided none OR more than 3 settings")

    print("\\begin{table}[H]")
    print("\\centering")
    print("\\resizebox{\\linewidth}{!}{\\begin{tabular}{"+textabular+"}")
    print(texheader)
    print(texdata,end="")
    print("\\end{tabular}}")
    print("\\caption{}")
    print("\\label{}")
    print("\\end{table}")

In [ ]:
# Set parameter values
alpha = 0.1
beta = 0.9
k = 10
N = 100
N_test = int(N*0.25) # Assume that the rest is used for training
par = 1
phi_div = phi.mod_chi2_cut
phi_dot = 1
time_limit_mosek = 10*60 #in seconds 
time_limit_alg = 10*60
numeric_precision = 1e-6 # To correct for floating-point math operations

In [ ]:
# Get generated data
data, data_nominal, data_test, data_train = generate_data(k, N, N_test)

In [ ]:
add_strategy = 'smallest_vio'
#add_strategy = 'N*(beta-lb)_smallest'
#add_strategy = 'random'
remove_cc = False
remove_nonbinding = False
runtime, x, obj, lb, true_prob, num_iter, Z_arr = solve_with_alg(add_strategy, remove_cc, remove_nonbinding, 
                                                               alpha, beta, par, phi_div, numeric_precision,
                                                               data_train, data_test, data_nominal, 
                                                               time_limit_mosek, time_limit_alg)

In [ ]:
# Plot final solution found by algorithm
name = add_strategy
save_plot = False
plot_type = "eps"
show_legend = True
plot_solution(name, data_train, Z_arr, x, obj, lb, save_plot, plot_type, show_legend)

In [ ]:
# Compute optimal solution with true probability constraint
prob_true = beta
[x_true, obj_true] = toymodel_true_prob(prob_true, k)
constr = data.dot(x_true)
p = np.array([len(constr[constr<=1])/N,len(constr[constr>1])/N])
r = phi_dot/(2*N)*scipy.stats.chi2.ppf(1-alpha, 1)
lb = lowbound(p, r, par, phi_div)
print(p)
print(lb)
print(obj_true)

In [ ]:
name = "TrueProb="+str(prob_true)
save_plot = False
plot_type = "eps"
show_legend = True

plot_solution(name, data, None, x_true, obj_true, lb, save_plot, plot_type, show_legend)

In [ ]:
# Determine optimal solution given data_test
runtime, opt_x, opt_sum_y, opt_obj, opt_lb = compute_opt_given_data(alpha, beta, par, phi_div, data_test, time_limit_mosek)

In [ ]:
# Plot optimal solution given data_test
name = 'Opt_given_test_data'
save_plot = False
plot_type = "eps"
show_legend = True
plot_solution(name, data_test, None, opt_x, opt_obj, opt_lb, save_plot, plot_type, show_legend)

In [ ]:
compute_calafiore_N_min(k, beta, alpha)

Following cells are used to obtain output and write to latex tables

In [6]:
headers = ['$k$', '$N$', '$N_{test}$', '$N_{cal}$', 'Obj.~(true prob.)', 'Obj.~(given test data)', 'Obj.~Alg.', 
           'Gap (\%)', 'LB', '\#iter', 'Time Alg.', 'Time MIP']

output_data = {}

# Variables parameter values
k_settings = [2, 5, 10]
N_settings = [100, 500, 1000]
N_test_settings = [0.25, 0.5, 0.75] #percentages of N

# Fixed parameter values
remove_cc = False
par = 1
phi_div = phi.mod_chi2_cut
phi_dot = 1
alpha = 0.1
beta = 0.9
time_limit_mosek = 10*60 #in seconds 
time_limit_alg = 10*60
numeric_precision = 1e-6 # To correct for floating-point math

# Alg settings
add_strategy = 'smallest_vio'
remove_cc = False
remove_nonbinding = False

for k in k_settings:
    N_cal = compute_calafiore_N_min(k, beta, alpha)
    
    for N in N_settings:
        for N_test_prop in N_test_settings:
            N_test = int(N_test_prop * N)
            
            data, data_nominal, data_test, data_train = generate_data(k, N, N_test)
            
            x_true, obj_true = toymodel_true_prob(beta, k)
            
            runtime_mip, x_mip, sum_y_mip, obj_mip, p_min = compute_opt_given_data(alpha, beta, par, phi_div, data_test, time_limit_mosek)               
            
            runtime_alg, x_alg, obj_alg, lb_alg, true_prob_x_alg, num_iter, Z_arr = solve_with_alg(add_strategy, remove_cc, remove_nonbinding, 
                                                                           alpha, beta, par, phi_div, numeric_precision,
                                                                           data_train, data_test, data_nominal, 
                                                                           time_limit_mosek, time_limit_alg)
            if runtime_mip is None:
                runtime_mip = 0
                obj_mip = 0
                gap_obj = 0
            else:
                gap_obj = 100*(obj_mip - obj_alg)/obj_mip
        
        
            output_data[(k, N, N_test)] = [N_cal, 
                                           f'{round(obj_true,3):.3f}',
                                           f'{round(obj_mip,3):.3f}',
                                           f'{round(obj_alg,3):.3f}', 
                                           f'{round(gap_obj,1):.1f}',
                                           f'{round(lb_alg,3):.3f}', 
                                           num_iter, 
                                           f'{round(runtime_alg, 0):.0f}',
                                           f'{round(runtime_mip, 0):.0f}']
    
write_output_to_latex(3, headers, output_data)

C:\Users\JSTARRE\Anaconda3\lib\site-packages\cvxpy\problems\problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


\begin{table}[H]
\centering
\resizebox{\linewidth}{!}{\begin{tabular}{lll|rrrrrrrrr}
$k$ & $N$ & $N_{test}$ & $N_{cal}$ & Obj.~(true prob.) & Obj.~(given test data) & Obj.~Alg. & Gap (\%) & LB & \#iter & Time Alg. & Time MIP\\
\hline
2 & 100 & 25 & 201 & 1.500 & 1.335 & 1.074 & 19.5 & 0.857 & 16 & 2 & 0 \\
2 & 100 & 50 & 201 & 1.500 & 1.800 & 1.762 & 2.1 & 0.901 & 12 & 2 & 0 \\
2 & 100 & 75 & 201 & 1.500 & 1.635 & 1.074 & 34.3 & 0.884 & 8 & 1 & 0 \\
2 & 500 & 125 & 201 & 1.500 & 1.429 & 1.421 & 0.6 & 0.920 & 84 & 9 & 0 \\
2 & 500 & 250 & 201 & 1.500 & 1.377 & 1.364 & 1.0 & 0.900 & 69 & 7 & 1 \\
2 & 500 & 375 & 201 & 1.500 & 1.400 & 1.364 & 2.5 & 0.907 & 40 & 4 & 1 \\
2 & 1000 & 250 & 201 & 1.500 & 1.254 & 1.250 & 0.3 & 0.905 & 192 & 21 & 0 \\
2 & 1000 & 500 & 201 & 1.500 & 1.322 & 1.309 & 1.0 & 0.902 & 159 & 18 & 5 \\
2 & 1000 & 750 & 201 & 1.500 & 1.322 & 1.309 & 1.0 & 0.903 & 67 & 7 & 14 \\
5 & 100 & 25 & 501 & 1.500 & 1.388 & 1.310 & 5.6 & 0.914 & 23 & 3 & 0 \\
5 & 100 & 50 & 501 & 